# 기상청 지점별(위.경도별) 일사량 자료 수집
* 기상청의 지점별 위도 및 경도별 일사량을 DATA화

## Column 설명
* 지점     : 각 기상대 지점의 이름
* 월       : 기상자료의 해당 월(month)
* 평균기온 : 월간 평균기온
* 평균풍속 : 월간 평균풍속(m/s)
* 평균운량 : 월간 평균 구름의 량
* 일조시간합 : 월간 일조시간 합계(hr)
* 천천일사합 : 월간 일사량 합계(MJ/m2)
* 위      도 : 지점의 위도
* 경      도 : 지점의 경도

## 참고자료
* 운량 : 10할 기준으로 맑음 : 0~2 , 구름조금 : 3~5, 구름많음 : 6~8, 흐림 : 9~10으로 정의  
* 일사량은 단위면적이 단위시간에 받는 일사에너지의 양으로 정의되며, 순간 복사량과 정해진 시간동안 단위면적이 받는 총열량인 복사량으로 표현됨  

In [135]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import re
import urllib.parse
import time

## 지점별 위치자료 입력

In [136]:
stninfo = pd.read_csv("./stninfo.csv",encoding='EUC-KR')
# 자료수집 년도 
search_year = "2018"

In [137]:
#C rome driver를 가상브라우져에 뛰우기 -----(1)
browser = webdriver.Chrome ("C:/webdriver/chromedriver")
# 3초 대기하기 ----(2)
browser.implicitly_wait(3)

## 지점별 기상청 자료수집

In [138]:
# 기상청 자료수집 프로그램 모듈
stninfo_len = len(stninfo)
#stninfo_len = 3
for i in range(0, stninfo_len) :
    jijum_name = str(stninfo.iloc[i,0])
    lat = stninfo.iloc[i,3]
    long = stninfo.iloc[i,4]
    jijum_name = "168_"+jijum_name
    print(stninfo.iloc[i,0],stninfo.iloc[i,1])
    url = "https://data.kma.go.kr/data/grnd/selectAsosRltmList.do"
    values = {
          'pgmNo':'36',
          'cmmnCdList':'F00501,F00502,F00503,F00512,F00513',
          'dataFormCd':'F00513',
          'dataReqstFileSn':"",
          'dataReqstSn':"",
          'dwldSetupPd':'0',
          'elementCds':'SFC01055001,SFC01056001,SFC01048001,SFC01057001,SFC01057003',
          'elementGroupSn':"",
          'elementGroupSns':'299,300,302,305',
          'endDt':'20190126,2018',
          'endHh':'00', 
          'endMt':'12',
          'endYear':search_year,
          'firstLoading':'',
          'lrgClssCd':'SFC',
          'mddlClssCd':'SFC01',
          'menuNo':'32',
          'pageIndex':'',
          'pageRowCount':'31',
          'schListCnt':'20',
          'schTotalCnt':'',
          'serviceSe':'F00102',
          'startDt':'20190120,2018',
          'startHh':'01',
          'startMt':'01',
          'startYear':search_year,
          'stnIds':jijum_name,
          'txtElementNm':'평균풍속,평균운량,평균기온,일조시간합,전천일사합',
          'txtStnNm':"추풍령",
          'upperCmmnCode':'F005'
          }
    params =  urllib.parse.urlencode(values)
    url = url + "?"+params
    browser.get(url)
    time.sleep(5)

    html = browser.page_source
    #가상브라우져에서 해당  Html파일을 추출한다.
    weather_data_temp = pd.read_html(html,header=0)
    weather_data_temp = pd.DataFrame(weather_data_temp[1])

    weather_data_temp['위도'] = lat
    weather_data_temp['경도'] = long
    weather_data_temp['시간'] = 1,2,3,4,5,6,7,8,9,10,11,12

    if i < 1 :
        weather_data = weather_data_temp
    else :
        weather_data = pd.concat([weather_data, weather_data_temp], axis=0)
    
browser.quit()

90 속초
93 북춘천
95 철원
98 동두천
99 파주
100 대관령
101 춘천
102 백령도
104 북강릉
105 강릉
106 동해
108 서울
112 인천
114 원주
115 울릉도
116 관악산
119 수원
121 영월
127 충주
129 서산
130 울진
131 청주
133 대전
135 추풍령
136 안동
137 상주
138 포항
140 군산
143 대구
146 전주
152 울산
155 창원
156 광주
159 부산
162 통영
164 무안
165 목포
168 여수
169 흑산도
170 완도
172 고창
174 순천
175 진도(첨찰산)
176 대구(기)
177 홍성
185 고산
187 성산
188 성산
189 서귀포
192 진주
201 강화
202 양평
203 이천
211 인제
212 홍천
214 삼척
216 태백
217 정선군
221 제천
226 보은
232 천안
235 보령
236 부여
238 금산
243 부안
244 임실
245 정읍
247 남원
248 장수
251 고창군
252 영광군
253 김해시
254 순창군
255 북창원
256 주암
257 양산시
258 보성군
259 강진군
260 장흥
261 해남
262 고흥
263 의령군
264 함양군
265 성산포
266 광양시
268 진도군
271 봉화
272 영주
273 문경
276 청송군
277 영덕
278 의성
279 구미
281 영천
283 경주시
284 거창
285 합천
288 밀양
289 산청
294 거제
295 남해


In [139]:
outputfile ='weather_data.csv'
weather_data.to_csv(outputfile, encoding='EUC-KR', index=False)